In [104]:
# To default to float division and print function.
from __future__ import (division, print_function)

# Core python libraries
import csv
import re
import math

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn

# Tensorflow and related.
import tensorflow

# For fitting streamlined tensor flow models.
import tflearn

# So you know when this code block finishes.
print ("Done")

Done


In [158]:
#TODO (max): make a library that does all this preprocessing
data = pandas.read_csv('data/train_large.csv')

# This is for use in exploring logistic regression. Need a binomial outcome variable
LABEL_COLUMN = "HighPayingJobs"
data[LABEL_COLUMN] = (data["SalaryNormalized"].apply(lambda x: x >= 30000)).astype(int)

print(len(data))

20000


In [170]:


X_train_index, X_test_index, Y_train, Y_test = sklearn.cross_validation.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.33, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

yb_train = X_train[LABEL_COLUMN]
yb_test= X_test[LABEL_COLUMN]

# Uncomment this next line if you want to check the data.
#print(data.head())

print ("Done")

Done


In [169]:
MIN_WORD_FREQUENCY = 3

count_vect = sklearn.feature_extraction.text.CountVectorizer(
  stop_words='english', min_df=MIN_WORD_FREQUENCY)

# Here I am count vectoring the full description field. But in theory
# any text field can be processed this way.
X_train_full_description_counts = count_vect.fit_transform(X_train['FullDescriptionWithTitle'])
X_test_full_description_counts = count_vect.transform(X_test['FullDescriptionWithTitle'])

print(X_train_full_description_counts.shape)
print(X_test_full_description_counts.shape)
#TODO (max): supress the VisibleDeprecationWarning here from numpy. It is being thrown by the package.

(13400, 14671)
(6600, 14671)


In [168]:
def mean_absolute_error_salary_scale(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        numpy.exp(y_test), numpy.exp(y_predicted))

# Guess the average. Create an empty vector of the desired shape.
average_guess = numpy.empty(Y_test.shape)
average_guess.fill(numpy.mean(Y_train))

mean_guess = average_guess[0]
print('Mean salary value in training set is  £{:10.2f}'.format(math.exp(mean_guess)))
average_guess_mae = mean_absolute_error_salary_scale(Y_test, average_guess)
print('Guess the average Mean Absolute Error: {:10.4f}'.format(average_guess_mae))

# SGD Needs normalized inputs
normalizer = sklearn.preprocessing.Normalizer(norm='l1')
X_train_norm = normalizer.fit_transform(X_train_full_description_counts.astype('float64'))
X_test_norm = normalizer.transform(X_test_full_description_counts.astype('float64'))

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(alpha=.005, penalty='l1', n_iter=100)
sgd.fit(X_train_norm, Y_train)
sgd_predictions = sgd.predict(X_test_norm)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))
#TODO (any): wonder why this is so inaccurate/wrong

Mean salary value in training set is  £  28152.20
Guess the average Mean Absolute Error: 12583.2907
SGDRegressor Mean Absolute Error: 13266.4945


In [171]:
# Will try and do a logistic regression to predict High paying jobs as per
# https://www.tensorflow.org/versions/r0.10/tutorials/wide/index.html
print(X_train[LABEL_COLUMN].value_counts())

0    7373
1    6027
dtype: int64


In [64]:
# feature_cols: A dict from feature column names to Tensors or SparseTensors.
# label: A Tensor containing the label column

# The keys of the feature_cols will be used to when construct columns in the next
# section. Because we want to call the fit and evaluate methods with different 
# data, we define two different input builder functions, train_input_fn and
# test_input_fn which are identical except that they pass different data to
# input_fn. Note that input_fn will be called while constructing the TensorFlow
# graph, not while running the graph. What it is returning is a representation
# of the input data as the fundamental unit of TensorFlow computations, a Tensor
# (or SparseTensor).

CONTINUOUS_COLUMNS = ["SalaryNormalized", "DescriptionLength"]

CATEGORICAL_COLUMNS = ["ContractTime", "ContractType"]

def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tensorflow.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tensorflow.SparseTensor.
  categorical_cols = {k: tensorflow.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols.items() + categorical_cols.items())
  # Converts the label column into a constant Tensor.
  label = tensorflow.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(X_train)

def eval_input_fn():
  return input_fn(X_test)


In [173]:
import tempfile

# This is a test. It should give 100% accuracy. Something is amiss.
salary_normalized = tensorflow.contrib.layers.real_valued_column(
    "SalaryNormalized")
# Real features.
description_length = tensorflow.contrib.layers.real_valued_column(
    "DescriptionLength")
contract_time = tensorflow.contrib.layers.sparse_column_with_hash_bucket(
    "ContractTime", hash_bucket_size=100)
contract_type = tensorflow.contrib.layers.sparse_column_with_hash_bucket(
    "ContractType", hash_bucket_size=100)



model_dir = tempfile.mkdtemp()
m = tensorflow.contrib.learn.LinearClassifier(feature_columns=[
        salary_normalized,
        #description_length, contract_time, contract_type
    ],
    #"""optimizer=tensorflow.train.FtrlOptimizer(
    #learning_rate=0.1,
    #l1_regularization_strength=1.0,
    #l2_regularization_strength=1.0)"""
  model_dir=model_dir)

In [174]:
m.fit(input_fn=train_input_fn, steps=2000)


LinearClassifier()

In [175]:
results = m.evaluate(input_fn=eval_input_fn, steps=100)
for key in sorted(results):
    print ("%s: %s" % (key, results[key]))

accuracy: 0.546364
eval_auc: 0.5
loss: 282.223


In [176]:
x, y = eval_input_fn()

# There is the old issue :D All 0 values :(
print(m.predict(input_fn=eval_input_fn))


[0 0 0 ..., 0 0 0]


In [68]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 1]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 2]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([1, 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [72]:
m = len(X_train)
x = X_train['SalaryNormalized']
y = 


print(m)
print(x)


670
703    25036
311    36480
722    39000
629    23520
0      25000
316    18240
706    22932
547    30263
872    23040
532    16041
477    35500
404    45120
172    16320
125    22617
394    17280
...
87     50000
458    17280
330    17875
214    16320
466    28500
121    10905
614    28000
20     25000
700    25250
71     22000
106    50000
270    15523
860    41280
435    17990
102    40000
Name: SalaryNormalized, Length: 670, dtype: int64


In [87]:

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(m/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})

SyntaxError: invalid syntax (<ipython-input-87-94f947ac16cb>, line 20)

In [98]:
import numpy as np

x = np.array(X_train['SalaryNormalized']).reshape([len(X_train),1])
y = np.array(X_train[LABEL_COLUMN]).reshape([len(X_train),1])

x_test = np.array(X_test['SalaryNormalized']).reshape([len(X_test),1])
y_test = np.array(X_test[LABEL_COLUMN]).reshape([len(X_test),1])

# DNN needs normalized inputs
normalizer = sklearn.preprocessing.Normalizer(norm='l1')
x_norm = normalizer.fit_transform(x.astype('float'))
x_test_norm = normalizer.transform(x_test.astype('float'))

In [99]:
print(x.shape)
print(x_norm.shape)

(670, 1)
(670, 1)


In [103]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(hidden_units=[10], n_classes=2)

# Fit model.
classifier.fit(x=x_norm, y=y, steps=2000)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(x=x_test_norm, y=y_test)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Accuracy: 0.669697
